In [1]:
# MNIST CNN
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.init
from torch.utils.data import DataLoader

In [2]:
device = 'cpu'
torch.manual_seed(333)

learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [3]:
mnist_train = dsets.MNIST(root='/tmp/pycharm_project_001/MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='/tmp/pycharm_project_001/MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

data_loader = DataLoader(dataset=mnist_train,
                         batch_size=batch_size,
                         shuffle=True,
                         drop_last=True)

In [4]:
class CNN(nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        # L1 Image shape = (?, 28, 28, 1)
        #       Conv     ->(?, 28, 28, 32)
        #       Pool     ->(?, 7, 7, 64)
        self.layer1 = nn.Sequential(nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
                                    nn.ReLU(),
                                    nn.MaxPool2d(kernel_size=2, stride=2)
                                    )
        # L2 Image shape = (?, 14, 14, 32)
        #       Conv     ->(?, 14, 14, 64)
        #       Pool     ->(?, 7, 7, 64)
        self.layer2 = nn.Sequential(nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
                                    nn.ReLU(),
                                    nn.MaxPool2d(kernel_size=2, stride=2)
                                    )
        # Final FC 7x7x64 inputs -> 10 outputs
        self.fc = nn.Linear(7 * 7 * 64, 10, bias=True)
        nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)     # Flatten them for FC
        out = self.fc(out)
        return out

In [5]:
model = CNN().to(device)

criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [6]:
total_batch = len(data_loader)

for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost /total_batch
    print('[Epoch : {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))
print('Learning finished')

[Epoch :    1] cost = 0.226260722
[Epoch :    2] cost = 0.0652078763
[Epoch :    3] cost = 0.04669521
[Epoch :    4] cost = 0.0373016931
[Epoch :    5] cost = 0.0312992521
[Epoch :    6] cost = 0.0278661102
[Epoch :    7] cost = 0.0221860092
[Epoch :    8] cost = 0.0179192089
[Epoch :    9] cost = 0.0161793996
[Epoch :   10] cost = 0.0141156623
[Epoch :   11] cost = 0.0111311125
[Epoch :   12] cost = 0.00987520162
[Epoch :   13] cost = 0.00915544853
[Epoch :   14] cost = 0.00773322256
[Epoch :   15] cost = 0.00628742157
Learning finished


In [7]:
with torch.no_grad():
    X_test = mnist_test.data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.targets.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy : ', accuracy.item())

Accuracy :  0.9830999970436096
